In [5]:
import pandas as pd
import pickle
from selenium import webdriver
import time
import furl
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
p = pickle.load(open("books_v9.pkl", "rb"))

In [4]:
driver = webdriver.Chrome(executable_path='/home/pybeast/chromedriver')

In [9]:
d = dict()
cnt = 0
for i in range(len(p))[0:1500]:
    cnt += 1
    try:
        amazon_url = "https://www.amazon.com/advanced-search/books"
        driver.get(amazon_url)
        try:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "field-isbn"))
            )
        except:
            pass
        
        isbn_input = driver.find_element_by_id("field-isbn")
        isbn = p.loc[i]['isbn_13']
        isbn_input.send_keys(isbn)
        search_button = driver.find_element_by_name("Adv-Srch-Books-Submit")
        search_button.click()
        time.sleep(3)
        
        d[p.loc[i]['isbn_13']] = dict()
        x = driver.find_element_by_xpath("//div[@class='sg-col sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20']")
        for j in x.find_elements_by_xpath(".//a[@class='a-size-base a-link-normal a-text-bold']"):
            url = furl.furl(j.get_attribute('href')).remove(args=True, fragment=True).url
            d[p.loc[i]['isbn_13']][j.text.strip()] = url
        for j in x.find_elements_by_xpath(".//a[@class='a-size-small a-link-normal']"):
            url = furl.furl(j.get_attribute('href')).remove(args=True, fragment=True).url
            d[p.loc[i]['isbn_13']][j.text.strip()] = url   
        print("SUCCESS #" + str(cnt))
    except:
        print("FAILED #" + str(cnt))

SUCCESS #1


In [ ]:
f = open("generated_file.txt", "w")
f.write(str(d))
f.close()